# Retrieve forcing for any CARAVAN catchment

In this notebook you will retrieve forcing from the Cravan dataset.The caravan dataset is a collection of streamflow and forcing data. <br>
Caravan was prepared by [Frederik Kratzert](https://doi.org/10.1038/s41597-023-01975-w), the forcing is based on the ERA5-Land model. The streamflow is from the USGS. <br>
To access it easily, it was stored [here](https://doi.org/10.4121/bf0eaf7c-f2fa-46f6-b8cd-77ad939dd350.v4) on the [OPenDAP](https://data.4tu.nl/info/about-your-data/netcdf-and-opendap) server from data.4TU.nl .<br>
This saves you from downloading and reading the whole dataset hosted on [zenodo](https://zenodo.org/records/6578598), instead only the necesarry data is downloaded. 

This notebook will show case how to run this for the HBV model. 

You have to supply the wanted basin_id. The shapefile will be downloaded automatically. 
Running the default will download the combined shapefile of all the catchments. 
Loading this combined shapefile and exploring it will show all availible catchments. 

In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# from rich import print

In [2]:
# general eWC
import ewatercycle
import ewatercycle.forcing
import ewatercycle.models
import ewatercycle.analysis

#### set up paths

In [3]:
path = Path.cwd()
forcing_path = path / "Forcing"

In [4]:
experiment_start_date = "1997-08-01T00:00:00Z"
experiment_end_date = "2005-09-01T00:00:00Z"
HRU_id = 3439000

### retrieve forcing

In [5]:
# pip install git+https://github.com/eWaterCycle/ewatercycle@main

In [6]:
# pip uninstall ewatercycle -y

In [7]:
import ewatercycle.forcing

In [8]:
print(ewatercycle.forcing.sources)

ForcingSources[
    "CaravanForcing",
    "DistributedMakkinkForcing",
    "DistributedUserForcing",
    "GenericDistributedForcing",
    "GenericLumpedForcing",
    "LumpedMakkinkForcing",
    "LumpedUserForcing"
]


In [9]:
import netCDF4 as ncf
ncf.__version__

'1.6.5'

In [12]:
caravan_forcing = ewatercycle.forcing.sources['CaravanForcing'].generate(start_time = experiment_start_date,
                                                                           end_time = experiment_end_date,
                                                                           directory = forcing_path / "Camels",
                                                                           basin_id = f"camels_0{HRU_id}",
                                                                             )

In [11]:
list(ds_caravan.data_vars.keys())

NameError: name 'ds_caravan' is not defined

In [ ]:
RENAME_ERA5 = {
    "total_precipitation_sum": "pr",
    "potential_evaporation_sum": "evspsblpot",
    "temperature_2m_mean": "tas",
    "temperature_2m_min": "tasmin",
    "temperature_2m_max": "tasmax",
    "streamflow": "Q",
}


In [ ]:
RENAME_ERA5.keys()

In [ ]:
ds_caravan

As you can see it will only download the required data variables needed for modeling

In [ ]:
ds_caravan.data_vars

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True, sharey=True)
ds_caravan['pr'].plot(ax=ax[0],label="P")
ds_caravan['evspsblpot'].plot(ax=ax[1],label="E",color="C1")
ax[0].set_title(f'{ds_caravan.name.values.astype(str)} \nid:"{ds_caravan.basin_id.values.astype(str)}"');
ax[1].set_title(None)
fig.tight_layout()

#### setup model 

In [ ]:
from ewatercycle.models import HBV

In [ ]:
model = HBV(forcing=caravan_forcing)

#### pass parameters

In [ ]:
s_0 = np.array([0,  100,  0,  5, 0])
par_0 = np.array([2,    0.8,  460,   1.5,    1.0,     4,    .4,   .04, 3])

In [ ]:
config_file, _ = model.setup(parameters=','.join([str(p) for p in par_0]),
                             initial_storage=','.join([str(s) for s in s_0]),
                             )

In [ ]:
model.initialize(config_file)

#### Run the model

In [ ]:
Q_m = []
time = []
while model.time < model.end_time:
    model.update()
    Q_m.append(model.get_value("Q"))
    time.append(model.time_as_datetime.date())

In [ ]:
hydro_data = pd.DataFrame(data=Q_m,index=time,columns=["model output"])
hydro_data['discharge observations'] = ds_caravan['Q'].to_pandas()

In [ ]:
hydro_data

Not a great result model wise but it runs and makes some sense!

In [ ]:
ewatercycle.analysis.hydrograph(hydro_data, reference='discharge observations');

In [ ]:
model.finalize()

### Basins

Using the shapefile we can look at the location of the basin on a map

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

I use geopandas:
```py
pip install geopandas
```
as using shapely/cartopy is a hastle

In [ ]:
import geopandas as gpd

In [ ]:
gdf_basin = gpd.read_file(caravan_forcing.shape)
gdf_basin_buffer = gdf_basin.buffer(1.5)

In [ ]:
gdf_basin

In [ ]:
# fig, ax = plt.subplots()
ax = plt.axes(projection=ccrs.PlateCarree())
country_borders = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_0_boundary_lines_land',
    scale='10m',
    facecolor='none')
ax.add_feature(country_borders, edgecolor='gray')

state_borders = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    facecolor='none')
ax.add_feature(state_borders, edgecolor='lightgray')
ax.add_feature(cfeature.COASTLINE, edgecolor='gray')
ax.add_feature(cfeature.OCEAN, edgecolor='lightblue')


gdf_basin.plot(ax=ax,facecolor="green")
gdf_basin_buffer.plot(ax=ax,facecolor="None", edgecolor="C3",zorder=10)

ax.set_title(f"{gdf_basin.loc[0,'gauge_id']} catchments")
ax.set_extent([-127.275,  -64.853, 25.864, 50.101], crs=ccrs.Geodetic())

plt.tight_layout()

##### and with the combined shapefile we see the all the basins

In [ ]:
gdf_all = gpd.read_file(caravan_forcing.directory / 'shapefiles' / 'combined.shp')

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE, edgecolor='gray')
gdf_all.plot(ax=ax,zorder=1,color="C0")
plt.tight_layout()

In [ ]:
gdf_all

In [ ]:
set(gdf_all['gauge_id'].apply(lambda x: x.split("_")[0]).values)

In [ ]:
import xarray as xr

In [ ]:
COMMON_URL = "ca13056c-c347-4a27-b320-930c2a4dd207"
OPENDAP_URL = f"https://opendap.4tu.nl/thredds/dodsC/data2/djht/{COMMON_URL}/1/"
SHAPEFILE_URL = (
    f"https://data.4tu.nl/file/{COMMON_URL}/bbe94526-cf1a-4b96-8155-244f20094719"
)


In [ ]:
def get_dataset(dataset) -> xr.Dataset:
    """Opens specified dataset from 4tu.nl opendap server.
        args:
            dataset:
                Choose from:    
                'camels', # US
                'camelsaus', # Australia
                'camelsbr', # Brazil
                'camelscl', # Chille 
                'camelsgb', # Great Britan
                'hysets',  # US
                'lamah', # US
    """
    return xr.open_dataset(f"{OPENDAP_URL}{dataset}.nc")

In [ ]:
def get_basin_id(dataset) -> list[str]:
    """
    args:
        dataset:
            Choose from:    
            'camels', 
            'camelsaus', 
            'camelsbr', 
            'camelscl', 
            'camelsgb', 
            'hysets', 
            'lamah'
    
    returns: 
        list of basin_ids of matching dataset
        
    """
    return [val.decode() for val in get_dataset(dataset).basin_id.values]